In [ ]:
%%time

import csv

def makeHeadfile(rawFileName='FlightDelays.csv', newFileName='FlightDelays_heads.csv', headSize=500, merge=False):
    data = []
    with open(rawFileName) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            data.append(row)
            line_count += 1
            if line_count > headSize:
                break
    with open(newFileName, mode='w', newline='') as head_file:
        head_writer = csv.writer(head_file, delimiter=',', quoting=csv.QUOTE_MINIMAL)
        head_writer.writerows(data)
    
    return data[0]

#save head of FlightDelays_1k_sample.csv file
raw_col = makeHeadfile(rawFileName='FlightDelays_1k_sample.csv',headSize=1000)

In [1]:
%%time

def dataLoad(baseFileName, addingFileName):
    '''
    data is a list of FlightDelays_head file.
    merge is a list of lists of AirFares file divided by its year and quaters to decrease search iteration.
    '''
    data = []
    merge=[]
    
    with open(baseFileName) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if row[31] != 1:
                data.append(row)
            
    with open(addingFileName) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            line_count = 0
            qt = []
            init = 0
            for row in csv_reader:
                if row[0] == '2018' and row[1] == '1':
                    qt.append(row)
                elif row[0] == '2018' and row[1] == '2':
                    if init==0:
                        merge.append(qt)
                        qt=[]
                        init = 1
                    qt.append(row)
                elif row[0] == '2018' and row[1] == '3':
                    if init==0:
                        merge.append(qt)
                        qt=[]
                        init = 1
                    qt.append(row)
                elif row[0] == '2018' and row[1] == '4':
                    if init==0:
                        merge.append(qt)
                        qt=[]
                        init = 1
                    qt.append(row)
                elif row[0] == '2019' and row[1] == '1':
                    if init==0:
                        merge.append(qt)
                        qt=[]
                        init = 1
                    qt.append(row)
                elif row[0] == '2019' and row[1] == '2':
                    if init==0:
                        merge.append(qt)
                        qt=[]
                        init = 1
                    qt.append(row)
                elif row[0] == '2019' and row[1] == '3':
                    if init==0:
                        merge.append(qt)
                        qt=[]
                        init = 1
                    qt.append(row)
                else:
                    AFindex = row
            merge.append(qt)
    return data, merge, AFindex


def addCol(data, colNames):
    for newcol in colNames:
        data = [row + [''] for row in data]
        data[0][-1] = newcol
    return data


def mergeFlightDelaysAndAirFares(data, merge):
    newData = []
    for data_row in data:
        if data_row[0] == '2018' and data_row[1] == '1':
            for i, merge_row in enumerate(merge[0]):
                if data_row[9] == merge_row[2] and data_row[10] == merge_row[4]:
                    data_row[-8:] = merge[0][i][-8:]
        elif data_row[0] == '2018' and data_row[1] == '2':
            for i, merge_row in enumerate(merge[1]):
                if data_row[9] == merge_row[2] and data_row[10] == merge_row[4]:
                    data_row[-8:] = merge[0][i][-8:]
        elif data_row[0] == '2018' and data_row[1] == '3':
            for i, merge_row in enumerate(merge[2]):
                if data_row[9] == merge_row[2] and data_row[10] == merge_row[4]:
                    data_row[-8:] = merge[0][i][-8:]
        elif data_row[0] == '2018' and data_row[1] == '4':
            for i, merge_row in enumerate(merge[3]):
                if data_row[9] == merge_row[2] and data_row[10] == merge_row[4]:
                    data_row[-8:] = merge[0][i][-8:]
        elif data_row[0] == '2019' and data_row[1] == '1':
            for i, merge_row in enumerate(merge[4]):
                if data_row[9] == merge_row[2] and data_row[10] == merge_row[4]:
                    data_row[-8:] = merge[0][i][-8:]            
        elif data_row[0] == '2019' and data_row[1] == '2':
            for i, merge_row in enumerate(merge[5]):
                if data_row[9] == merge_row[2] and data_row[10] == merge_row[4]:
                    data_row[-8:] = merge[0][i][-8:]
        elif data_row[0] == '2019' and data_row[1] == '3':
            for i, merge_row in enumerate(merge[6]):
                if data_row[9] == merge_row[2] and data_row[10] == merge_row[4]:
                    data_row[-8:] = merge[0][i][-8:]
        
            
        
        newData.append(data_row)
    return newData

def removeOtherNoise(data, noise_list):
    rm = []
    ndata = []
    for noise in noise_list:
        for i, att in enumerate(mergedData[0]):
            if att == noise:
                break
        for line in mergedData:
            line.pop(i)
            
    return mergedData

Wall time: 0 ns


In [5]:
%%time
import csv
baseFileName = 'FlightDelays_heads.csv'
addingFileName = 'AirFares.csv'

#Load csv files of FlightDelays_heads and AirFares
data, merge, AFindex = dataLoad(baseFileName, addingFileName)


#make new columns to FlightDelays_heads from AirFares
newCols = AFindex[6:]
data = addCol(data, newCols)

#Merge fare information from AirFares to FlightDelays_head file by searching their airport information.
mergedData = mergeFlightDelaysAndAirFares(data, merge)


Wall time: 1.1 s


In [7]:
%%time
import pandas as pd
import numpy as np

df_MG = pd.DataFrame.from_records(mergedData[1:], columns=mergedData[0])
df_MG = df_MG[df_MG.CANCELED == '0']

inputX = ['YEAR', 'QUARTER', 'MONTH','DAY_OF_MONTH','DAY_OF_WEEK','CARRIER','FL_NUM','Route','ORIGIN', 'DEST',
       'CRS_DEP_TIME', 'CRS_ARR_TIME', 'CRS_ELAPSED_TIME', 'DISTANCE', 'PASSENGERS', 'EMPFTE', 'NET_INCOME',
        'OP_REVENUES', 'fare', 'carrier_lg', 'large_ms', 'fare_lg', 'ARR_DELAY_GROUP']

df_MG = df_MG[inputX]
df_MG = df_MG.replace(np.nan, 0)
df_MG = df_MG.replace('', 0)



Wall time: 15.9 ms


In [8]:
def codeHash(code):
    # Example: variable="ABC"=>26^2*1+26*2+3=731
    sum0 = 0
    loop = 1
    for char in code[::-1]:
        
        sum0+=(ord(char)-ord('A')+1)*(loop)
        loop*=26
    #they are too large or small
    return float(sum0)

def typeDef(df_MG):
    df_MG = df_MG.astype({'YEAR' : float})
    df_MG = df_MG.astype({'QUARTER' : float})
    df_MG = df_MG.astype({'MONTH' : float})
    df_MG = df_MG.astype({'DAY_OF_MONTH' : float})
    df_MG = df_MG.astype({'DAY_OF_WEEK' : float})
    df_MG = df_MG.astype({'CARRIER': str})
    df_MG = df_MG.astype({'FL_NUM': float})
    df_MG = df_MG.astype({'Route': float})
    df_MG = df_MG.astype({'ORIGIN': str})
    df_MG = df_MG.astype({'DEST': str})
    df_MG = df_MG.astype({'CRS_DEP_TIME': str})
    df_MG = df_MG.astype({'CRS_ARR_TIME': str})
    df_MG = df_MG.astype({'CRS_ELAPSED_TIME': float})
    df_MG = df_MG.astype({'DISTANCE': float})
    df_MG = df_MG.astype({'PASSENGERS': float})
    df_MG = df_MG.astype({'EMPFTE': float})
    df_MG = df_MG.astype({'NET_INCOME': float})
    df_MG = df_MG.astype({'OP_REVENUES': float})
    df_MG = df_MG.astype({'fare': float})
    df_MG = df_MG.astype({'carrier_lg': str})
    df_MG = df_MG.astype({'large_ms': float})
    df_MG = df_MG.astype({'fare_lg': float})
    df_MG = df_MG.astype({'ARR_DELAY_GROUP': float})
    return df_MG

def changeVal(df_MG):
    df_MG['CARRIER'] = df_MG['CARRIER'].apply(codeHash)
    df_MG['ORIGIN'] = df_MG['ORIGIN'].apply(codeHash)
    df_MG['DEST'] = df_MG['DEST'].apply(codeHash)
    df_MG['carrier_lg'] = df_MG['carrier_lg'].apply(codeHash)

    f = lambda x: (float(x[:2])*60+float(x[2:])) if (len(x) > 3) else ((float(x[0])*60+float(x[1:])) if (len(x) <=3 and len(x) >= 2) else 0)
    df_MG['CRS_DEP_TIME'] = df_MG['CRS_DEP_TIME'].apply(f)
    df_MG['CRS_ARR_TIME'] = df_MG['CRS_ARR_TIME'].apply(f)
    return df_MG

def fareFill(df_MG):
    '''
    need to find a way how to fill in the empty values
    '''
    
    return df_MG

In [10]:
%%time
df_MG = changeVal(typeDef(df_MG))
df_MG.to_csv(r'data.csv', index = True)

Wall time: 95.3 ms


Tablueau

In [ ]:
# import pantab
# pantab.frame_to_hyper(df_MG, "MG1.hyper", table="the_table") 
# df_MG

Pandas Profiling  
Doesn't work for now

In [ ]:

# from pandas_profiling import ProfileReport
# profile = ProfileReport(df_MG)